## This notebook is a script for automatically import csv files to an database via Python API.
### By this script, we attempt to generate a new table in database schema and import csv files all automatically.

In [1]:
import sqlalchemy
import os

import pandas as pd
import numpy as np

print('Library imported.')

C:\Users\lawrence123\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lawrence123\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\lawrence123\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Library imported.


### List all CSV files to import

In [2]:
import glob

FILE_PATH = 'D:/AISP測試用檔案/simulate_platform/data'

df = pd.read_csv(os.path.join(FILE_PATH, 'spam.csv'))
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [33]:
import psycopg2

# Create connection to DB

TABLE_NAME = 'AI_testdata'

SETTING_PATH = 'D:/AISP測試用檔案/simulate_platform/notebook'
with open(os.path.join(SETTING_PATH, 'setting.txt'), 'r') as file:
    settings = file.readlines()
    for line in settings:
        value = line.split('=')[1]
        if 'dbname' in line:
            DATABASE = value.strip() # strip remove \n trailing whitespace
        elif 'user' in line:
            USER = value.strip()
        elif 'pwd' in line:
            PWD = value.strip()
        elif 'port' in line:
            PORT = int(value)
        elif 'host' in line:
            HOST = value.strip()
    

conn = psycopg2.connect(dbname = DATABASE, 
                             user = USER,
                             host = HOST,
                             port = PORT,
                             password = PWD
                             )
    
print('Connected to DB successful!')

Connected to DB successful!


In [32]:
if not conn.closed:
    print('Database is connected!')

Database is connected!


### Note: Auto-generate COLUMN type from CSV would just create a general purpose script. It could not be customized for various requiement of data types. This method that auto-generate CREATE TABLE and LOAD DATA is just for convenient to load multiple files to DATABASE faster.

In [34]:
from sqlalchemy import create_engine

# Load csv files to database and create table


sqlengine = create_engine(f"postgresql+psycopg2://{USER}:{PWD}@{HOST}:{PORT}/{DATABASE}")
engine_conn = sqlengine.connect()


# Start import data to table
try:
    frame = df.to_sql(
            TABLE_NAME,
            con = engine_conn,
            if_exists = 'replace'
        )

except Exception as E:
        print('Error occurs: \n', E)
else:
        print('Successfully create new table and import data!')

conn.close() # Close the connection


Successfully create new table and import data!
